In [1]:
from importlib import reload
def reload_all():
    import tsm; reload(tsm)
    import _keys; reload(_keys)
    import blizzard; reload(blizzard)
    import config; reload(config)
    import kvstore; reload(kvstore)
    import snapshot; reload(snapshot)
    import elemental_arbitrage; reload(elemental_arbitrage)
    import bliz_tsm_join; reload(bliz_tsm_join)
    import crafting; reload(crafting)
    import kvstore; reload(kvstore)
    import crafting; reload(crafting)
    import procurement; reload(procurement)
reload_all()
from itables import show

In [2]:
from snapshot import SnapshotProcessor
from tsm import auction_house_snapshot
from config import tsm_region_id
from config import tsm_realm_id
from config import tsm_ah_id
from config import tsm_cache_dir

def tsm_ah_snapper():
    return auction_house_snapshot(tsm_region_id, tsm_realm_id, tsm_ah_id)

tsm_ah_snap = SnapshotProcessor(tsm_ah_snapper, cache_dir=tsm_cache_dir)
tsm_ah = tsm_ah_snap.get(max_age_seconds=3000)

In [3]:
from snapshot import SnapshotProcessor
from blizzard import auction_data
from blizzard import auction_summary
from config import blizzard_realm_id
from config import blizzard_ah_id
from config import blizzard_cache_dir

def bliz_ah_snapper():
    return auction_data(blizzard_realm_id, blizzard_ah_id)

bliz_ah_snap = SnapshotProcessor(bliz_ah_snapper, cache_dir=blizzard_cache_dir)
bliz_ah = bliz_ah_snap.get(max_age_seconds=3000)

In [4]:
from kvstore import InefficientKVStore
from blizzard import ItemLookup
from blizzard import collapse_languages
from config import blizzard_item_cache
from config import blizzard_item_reverse_cache

items = ItemLookup(
    InefficientKVStore(blizzard_item_cache),
    InefficientKVStore(blizzard_item_reverse_cache),
)

In [5]:
from bliz_tsm_join import item_info_getter
item_info = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=3000)
item_info_current = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=0)

In [6]:
from cytoolz import groupby
import pandas as pd

df_sales = pd.read_csv("Accounting_Earthfury_sales.csv")
df_sales["mode"] = "sold"
df_purchases = pd.read_csv("Accounting_Earthfury_purchases.csv")
df_purchases["mode"] = "purchased"
df_expired = pd.read_csv("Accounting_Earthfury_expired.csv")
df_expired["mode"] = "expired"
df_canceled = pd.read_csv("Accounting_Earthfury_canceled.csv")
df_canceled["mode"] = "canceled"
df_all = pd.concat([df_sales, df_purchases, df_expired, df_canceled])

entries = list(df_all.transpose().to_dict().values())
grouped = groupby(lambda entry: entry["itemString"], entries)
names = {entry["itemString"]: entry["itemName"] for entry in entries}
nets = {
    k: sum(
        (
            vv["price"]*vv["quantity"] if vv["mode"] == "sold" else
            -vv["price"]*vv["quantity"] if vv["mode"] == "purchased" else
            0
        ) for vv in v
    )
    for (k, v) in grouped.items()
}
ah_events = {k: [vv["mode"] for vv in v if vv["source"] != "Vendor"] for (k, v) in grouped.items()}
sale_rate = {
    k: v.count("sold") / (v.count("sold") + v.count("expired") + v.count("canceled"))
    for (k, v) in ah_events.items()
    if v.count("sold") > 1
}
agg = {k: {"name": names[k], "net": nets[k], "rate": sale_rate[k], "num": len(ah_events[k])} for k in sale_rate}
show(pd.DataFrame(sorted(
    [v for v in agg.values() if v["num"] > 4],
    key=lambda entry: entry["net"],
    reverse=True,
)))

<ipython-input-6-e5b0eba07282>:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  entries = list(df_all.transpose().to_dict().values())


In [7]:
import pickle
from cytoolz import get as pick_from_dict
from cytoolz import sliding_window
import glob
import itertools
from blizzard import auction_summary
import re
import numpy as np
import datetime
np.set_printoptions(suppress=True)

_names = """
flask of the frost wyrm
flask of endless rage
flask of stoneblood
titanium bar
indestructible potion
eternal shadow
oily blackmouth
elixir of mighty thoughts
frostweave cloth
wildvine
felcloth
lesser healing potion
runic mana potion
greater planar essence
netherweave cloth
greater healing potion
crystallized water
purple lotus
bruiseweed
arcane dust
tiger lily
crystallized fire
major healing potion
potion of speed
potion of wild magic
lichbloom
pygmy oil
adder's tongue
imbued vial
icethorn
frost lotus
enchanted vial
goldclover
crystallized life
eternal life
deadnettle
elixir of accuracy
lesser planar essence
eternal fire
eternal water
chaotic skyflare diamond
skyflare diamond
potent monarch topaz
monarch topaz
runed scarlet ruby
scarlet ruby
purified twilight opal
twilight opal
potion of speed
potion of wild magic
lichbloom
frostweave cloth
infinite dust
greater cosmic essence
dream shard
eternal belt buckle
titansteel bar
titansteel spellblade
mageweave cloth
rough stone
copper bar
coarse stone
linen cloth
bronze bar
heavy stone
moss agate
wool cloth
medium leather
steel bar
solid stone
mithril bar
mageweave cloth
dense stone
thorium bar
runecloth
fel iron bar
mote of earth
mote of fire
netherweave cloth
adamantite bar
cobalt bar
frostweave cloth
crystallized water
crystallized earth
crystallized fire
borean leather
eternal shadow
saronite bar
liferoot
blackmouth oil
briarthorn
bruiseweed
elixir of greater defense
fadeleaf
goldclover
goldthorn
greater healing potion
imbued vial
kingsblood
lesser invisibility potion
lichbloom
liferoot
major troll's blood elixir
nature protection potion
oily blackmouth
runic mana potion
stranglekelp
swiftness potion
swiftthistle
swim speed potion
wild steelbloom
elixir of water walking
elixir of mighty thoughts
""".strip().splitlines()

desired_names = list(set(_names))
desired_ids = [items.get_id(name) for name in desired_names if not name.startswith("#")]


date_re = re.compile(r"202.-..-..T..-..-..")
def _timestamp_from_name(name):
    cap = date_re.search(name).group(0)
    return datetime.datetime.strptime(cap, "%Y-%m-%dT%H-%M-%S").timestamp()


by_item = {}
timeseries = []

for snapshot in sorted(glob.glob("bliz-ah/*")):
    with open(snapshot, "rb") as f:
        entries = pickle.load(f).values()
        timeseries.extend(
            [
                {"timestamp": _timestamp_from_name(snapshot), "item_name": items.get_name(item_id), **auction_summary(item_data)}
                for (item_id, item_data) in zip(desired_ids, entries)
            ]
        )
        for (item_id, item_data) in zip(desired_ids, entries):
            name = items.get_name(item_id)
            by_item[name] = by_item.get(name, []) + [{"timestamp": date_re.search(snapshot).group(0), "item_name": items.get_name(item_id), **auction_summary(item_data)}]

def timeseries_of(key, default=np.nan):
    def _timeseries_of(item_name):
        return np.array([v.get(key, default) for v in by_item[item_name]])
    return _timeseries_of

def omit_outliers(series, sigma=2, window=5):
    mavg = [np.average(w) for w in sliding_window(window, series)]
    std = np.std(mavg)
    return np.array([x if np.abs(x) < sigma*std else np.nan for x in mavg])

def avg_finite(arr):
    return np.average(arr[np.isfinite(arr)])

def min_series(x):
    return omit_outliers(timeseries_of("avg_sell")(x), sigma=0.5, window=10)

In [8]:
import plotly.express as px
for yvalue in ["num", "min", "p50", "d_num", "d_min", "d_p50"]:
    df = pd.DataFrame(timeseries)
    df = df.sort_values(by="timestamp")

    if yvalue.startswith("d_"):
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue[2:])
        dfg = dfg.diff().div(dfg.index.to_series().diff(), axis=0)
    else:
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue)

    fig = px.scatter(dfg, title=yvalue, log_y=False)
    fig.write_html(f"{yvalue}.html")

In [9]:
def item_infos(items):
    return pd.DataFrame([item_info(i) for i in items])

In [10]:
ii = item_infos(_names)
show(ii)

In [11]:
from crafting import Recipes
from crafting import purchase_price
from crafting import crafting_price
from crafting import gold


r = Recipes(items)
i = r.ingredients

def buy(item, price_method=gold("realm_market_value")):
    return purchase_price(item_info, item, price_method)


def buy_now(item):
    return buy(item, price_method=gold("min"))


def buy_market(item):
    return buy(item, price_method=gold("realm_market_value"))


recipe_listing = """
2 flask of the frost wyrm
5 icethorn + 5 lichbloom + frost lotus + enchanted vial

2 flask of endless rage
7 lichbloom + 3 goldclover + frost lotus + enchanted vial

2 flask of stoneblood
7 lichbloom + 3 crystallized life + frost lotus + enchanted vial

firecracker salmon
glacial salmon + northern spices

elixir of greater agility
sungrass + goldthorn + crystal vial

titanium bar
8 saronite bar

titanium bar
2 titanium ore

titansteel bar
3 titanium bar + eternal fire + eternal earth + eternal shadow

indestructible potion
2 icethorn + imbued vial

saronite bar
2 saronite ore

potion of speed
2 adder's tongue + pygmy oil + imbued vial

potion of wild magic
2 lichbloom + pygmy oil + imbued vial

titansteel spellblade
6 titansteel bar + 6 infinite dust + 2 frozen orb

titansteel bar
3 titanium bar + eternal fire + eternal earth + eternal shadow

glyph of wild growth
resilient parchment + ethereal ink

spellweave robe
8 spellweave + 6 bolt of imbued frostweave + eternium thread + frozen orb

spellweave
bolt of imbued frostweave + 2 eternal fire

bolt of imbued frostweave
2 bolt of frostweave + 2 infinite dust

bolt of frostweave
5 frostweave cloth

earthsiege diamond
dark jade + huge citrine + eternal fire

skyflare diamond
bloodstone + chalcedony + eternal air

noise machine
2 froststeel tube + 2 overcharged capacitor + 8 handful of cobalt bolts

froststeel tube
8 cobalt bar + crystallized water

overcharged capacitor
4 cobalt bar + crystallized earth

handful of cobalt bolts
2 cobalt bar

cobalt bar
cobalt ore

mechanized snow goggles
8 saronite bar + 2 borean leather + eternal shadow

saronite bar
2 saronite ore

wormhole generator: northrend
8 titanium bar + 2 eternal shadow + 2 eternal fire + 2 eternal water + 2 eternal air

pygmy oil
pygmy suckerfish

runic mana potion
2 lichbloom + goldclover + imbued vial

greater healing potion
liferoot + kingsblood + leaded vial

major troll's blood elixir
liferoot + bruiseweed + leaded vial

nature protection potion
liferoot + stranglekelp + leaded vial

swiftness potion
swiftthistle + briarthorn + leaded vial

swim speed potion
swiftthistle + blackmouth oil + leaded vial

blackmouth oil
2 oily blackmouth

elixir of greater defense
wild steelbloom + goldthorn + leaded vial

lesser invisibility potion
wild steelbloom + fadeleaf + leaded vial

great feast
chunk o' mammoth + shoveltusk flank + worm meat + 2 chilled meat

fish feast
2 musselback sculpin + 2 glacial salmon + 2 nettlefish + northern spices

2 flask of pure mojo
7 icethorn + 3 pygmy oil + frost lotus + enchanted vial

scroll of enchant weapon - mighty spellpower
30 infinite dust + 6 dream shard + 6 abyss crystal

barbaric bracers
8 heavy leather + 2 cured heavy hide + 4 small lustrous pearl + raptor hide + 4 large fang

earthen leg armor
2 arctic fur + 4 eternal earth + frozen orb

frosthide leg armor
2 arctic fur + 2 nerubian chitin + frozen orb

2 flask of relentless assault
7 terocone + 3 mana thistle + fel lotus + imbued vial

glacial bag
4 moonshroud + 4 ebonweave + eternium thread

#moonshroud
#bolt of imbued frostweave + 2 eternal life

#ebonweave
#bolt of imbued frostweave + 2 eternal shadow

frostweave bag
6 bolt of imbued frostweave + 2 eternium thread

primal might
primal earth + primal water + primal fire + primal air + primal mana

primal earth
10 mote of earth

primal life
10 mote of life

primal water
10 mote of water

primal air
10 mote of air

primal fire
10 mote of fire

primal mana
10 mote of mana

primal shadow
10 mote of shadow
"""

reversible_recipes = """
eternal shadow
10 crystallized shadow

eternal water
10 crystallized water

eternal fire
10 crystallized fire

eternal air
10 crystallized air

eternal life
10 crystallized life

greater planar essence
3 lesser planar essence

greater cosmic essence
3 lesser cosmic essence
"""

vendor = {
    "wild spineleaf": 0.45,
    "enchanted vial": 0.90,
    "imbued vial": 0.36,
    "leaded vial": 0.016,
}


r_out = None
for line in recipe_listing.splitlines():
    if line.strip() and not line.strip().startswith("#"):
        if r_out:
            r.recipe_from_strings(r_out, line.strip())
            r_out = None
        else:
            r_out = line.strip()

            
r_out = None
for line in reversible_recipes.splitlines():
    if line.strip() and not line.strip().startswith("#"):
        if r_out:
            r.recipe_from_strings(r_out, line.strip())
            r.recipe_from_strings(line.strip(), r_out)
            r_out = None
        else:
            r_out = line.strip()


from procurement import ProcurementPlanner
from procurement import topk_procurements
import procurement
from cytoolz import groupby
from cytoolz import valmap
pp = ProcurementPlanner(items, item_info, r, vendor, approaches=["ah_buy_now", "craft", "vendor"])
pp_market = ProcurementPlanner(items, item_info, r, vendor, approaches=["ah_buy_market", "craft", "vendor"])

In [12]:
everybody = """
20 indestructible potion
20 potion of speed
20 potion of wild magic
20 flask of stoneblood
20 flask of the frost wyrm
20 flask of pure mojo
20 flask of endless rage
20 skyflare diamond
20 earthsiege diamond
""".strip().splitlines()
best_procurements_market = procurement.And.flat(
    min(procurement.dnf(pp_market.obtain(i(x.strip()))), key=procurement.cost)
    for x in everybody
)
best_procurements_now = procurement.And.flat(
    min(procurement.dnf(pp.obtain(i(x.strip()))), key=procurement.cost)
    for x in everybody
)

In [13]:
best_procurements = best_procurements_now
groupby(
    lambda x: type(x),
    [
        op.sum(v)
        for ((op, name), v)
        in groupby(
            lambda y: (type(y), y.item.pure()[0]),
            best_procurements
        ).items()
    ],
)

{procurement.Craft: [Craft(item=20*indestructible potion, ingredients=20.0*imbued vial + 40.0*icethorn),
  Craft(item=20*potion of speed, ingredients=40.0*adder's tongue + 20.0*imbued vial + 20.0*pygmy oil),
  Craft(item=70.0*pygmy oil, ingredients=70.0*pygmy suckerfish),
  Craft(item=20*potion of wild magic, ingredients=20.0*imbued vial + 20.0*pygmy oil + 40.0*lichbloom),
  Craft(item=20*flask of stoneblood, ingredients=30.0*crystallized life + 10.0*enchanted vial + 10.0*frost lotus + 70.0*lichbloom),
  Craft(item=20*flask of the frost wyrm, ingredients=10.0*frost lotus + 10.0*enchanted vial + 50.0*icethorn + 50.0*lichbloom),
  Craft(item=20*flask of pure mojo, ingredients=10.0*frost lotus + 30.0*pygmy oil + 10.0*enchanted vial + 70.0*icethorn),
  Craft(item=20*flask of endless rage, ingredients=10.0*frost lotus + 30.0*goldclover + 10.0*enchanted vial + 70.0*lichbloom),
  Craft(item=20*skyflare diamond, ingredients=20.0*eternal air + 20.0*chalcedony + 20.0*bloodstone),
  Craft(item=20

In [14]:
[(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("swiftness potion")))]

[(8.3093, And(AHBuyMarket(item=swiftness potion, gold=8.3093),)),
 (3.3307,
  And(Craft(item=swiftness potion, ingredients=leaded vial + briarthorn + swiftthistle), VendorBuy(item=leaded vial, who='vendor', where='somewhere', gold=0.016), AHBuyMarket(item=briarthorn, gold=0.9822), AHBuyMarket(item=swiftthistle, gold=2.3325)))]

In [15]:
[(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("swim speed potion")))]

[(0.89, And(AHBuyMarket(item=swim speed potion, gold=0.89),)),
 (4.5428999999999995,
  And(Craft(item=swim speed potion, ingredients=leaded vial + swiftthistle + blackmouth oil), VendorBuy(item=leaded vial, who='vendor', where='somewhere', gold=0.016), AHBuyMarket(item=swiftthistle, gold=2.3325), AHBuyMarket(item=blackmouth oil, gold=2.1944))),
 (3.2365,
  And(Craft(item=swim speed potion, ingredients=leaded vial + swiftthistle + blackmouth oil), VendorBuy(item=leaded vial, who='vendor', where='somewhere', gold=0.016), AHBuyMarket(item=swiftthistle, gold=2.3325), Craft(item=blackmouth oil, ingredients=2.0*oily blackmouth), AHBuyMarket(item=2.0*oily blackmouth, gold=0.888)))]

In [16]:
sorted([(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("elixir of water walking")))])

[(1.9131, And(AHBuyMarket(item=elixir of water walking, gold=1.9131),))]

In [17]:
sorted([(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("flask of pure mojo")))])

[(10.8076,
  And(Craft(item=flask of pure mojo, ingredients=0.5*enchanted vial + 0.5*frost lotus + 1.5*pygmy oil + 3.5*icethorn), VendorBuy(item=0.5*enchanted vial, who='vendor', where='somewhere', gold=0.9), AHBuyMarket(item=0.5*frost lotus, gold=3.089), AHBuyMarket(item=1.5*pygmy oil, gold=3.7627500000000005), AHBuyMarket(item=3.5*icethorn, gold=3.05585))),
 (11.605,
  And(Craft(item=flask of pure mojo, ingredients=0.5*enchanted vial + 0.5*frost lotus + 1.5*pygmy oil + 3.5*icethorn), VendorBuy(item=0.5*enchanted vial, who='vendor', where='somewhere', gold=0.9), AHBuyMarket(item=0.5*frost lotus, gold=3.089), Craft(item=1.5*pygmy oil, ingredients=1.5*pygmy suckerfish), AHBuyMarket(item=1.5*pygmy suckerfish, gold=4.56015), AHBuyMarket(item=3.5*icethorn, gold=3.05585))),
 (12.6014, And(AHBuyMarket(item=flask of pure mojo, gold=12.6014),)),
 (23.437,
  And(Craft(item=flask of pure mojo, ingredients=0.5*enchanted vial + 0.5*frost lotus + 1.5*pygmy oil + 3.5*icethorn), AHBuyMarket(item=0.5*

In [18]:
sorted([(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("scroll of enchant weapon - mighty spellpower")))])

[(1217.5884,
  And(Craft(item=scroll of enchant weapon - mighty spellpower, ingredients=30.0*infinite dust + 6.0*dream shard + 6.0*abyss crystal), AHBuyMarket(item=30.0*infinite dust, gold=51.165), AHBuyMarket(item=6.0*dream shard, gold=125.09040000000002), AHBuyMarket(item=6.0*abyss crystal, gold=1041.333))),
 (1712.1391,
  And(AHBuyMarket(item=scroll of enchant weapon - mighty spellpower, gold=1712.1391),))]

In [19]:
hodir_badges_needed = (21000-4425)/50

In [20]:
hodir_badges_needed

331.5

In [21]:
item_info("barbaric bracers")

{'id': 18948,
 'name': 'barbaric bracers',
 'num_auctions': 9999,
 'quantity': 9999,
 'weight_sell': 0.7945,
 'avg_sell': 0.7945,
 'max': 0.7945,
 'p80': 0.7945,
 'p50': 0.7945,
 'p20': 0.7945,
 'wp80': 0.7945,
 'wp50': 0.7945,
 'wp20': 0.7945,
 'min': 0.7945,
 'realm_market_value': 127.3891,
 'realm_historical': 237.0,
 'region_historical': 53.3284,
 'region_avg_sale_price': 69.7007,
 'sale_pct': 29.7,
 'sold_per_day': 6,
 'headroom': -9978,
 'market_skew_pct': -46.25,
 'auction_skew_pct': -100.0}

In [22]:
sorted([(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("barbaric bracers")))])

[(30.9794,
  And(Craft(item=barbaric bracers, ingredients=4.0*large fang + 2.0*cured heavy hide + 4.0*small lustrous pearl + raptor hide + 8.0*heavy leather), AHBuyMarket(item=4.0*large fang, gold=12.0068), AHBuyMarket(item=2.0*cured heavy hide, gold=4.1768), AHBuyMarket(item=4.0*small lustrous pearl, gold=10.2868), AHBuyMarket(item=raptor hide, gold=0.0298), AHBuyMarket(item=8.0*heavy leather, gold=4.4792))),
 (127.3891, And(AHBuyMarket(item=barbaric bracers, gold=127.3891),))]

In [23]:
sorted([(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("earthen leg armor")))])

[(166.24790000000002,
  And(Craft(item=earthen leg armor, ingredients=4.0*eternal earth + 2.0*arctic fur + frozen orb), AHBuyMarket(item=4.0*eternal earth, gold=38.2232), AHBuyMarket(item=2.0*arctic fur, gold=53.1754), AHBuyMarket(item=frozen orb, gold=74.8493))),
 (298.6705, And(AHBuyMarket(item=earthen leg armor, gold=298.6705),))]

In [24]:
sorted([(procurement.cost(x), x) for x in procurement.dnf(pp_market.obtain(i("frosthide leg armor")))])

[(130.7841,
  And(Craft(item=frosthide leg armor, ingredients=2.0*arctic fur + 2.0*nerubian chitin + frozen orb), AHBuyMarket(item=2.0*arctic fur, gold=53.1754), AHBuyMarket(item=2.0*nerubian chitin, gold=2.7594), AHBuyMarket(item=frozen orb, gold=74.8493))),
 (211.9681, And(AHBuyMarket(item=frosthide leg armor, gold=211.9681),))]

In [25]:
# hateful PVP gear
sum(
[
    4520, # gloves
    7700, # helm
    6110, # spaulders
    7700, # legguards
    7700, # robes
    3990, # idol
]
)


sum(
[
    10850, # belt
    #10850, # boots
    6895, # armwraps
    8295, # pendant
    #8295, # cloak
    #8295, # band
]
)

26040

In [26]:
options = procurement.dnf(pp_market.obtain(i("glacial bag")))
(c_, r_) = min([(procurement.cost(x), x) for x in options], key=lambda x: x[0])
(c_, list(r_))

(1402.2626,
 [Craft(item=glacial bag, ingredients=4.0*ebonweave + 4.0*moonshroud + eternium thread),
  AHBuyMarket(item=4.0*ebonweave, gold=728.3392),
  AHBuyMarket(item=4.0*moonshroud, gold=670.4236),
  AHBuyMarket(item=eternium thread, gold=3.4998)])

In [27]:
options = procurement.dnf(pp_market.obtain(i("glacial bag")))
(c_, r_) = min([(procurement.cost(x), x) for x in options], key=lambda x: x[0])
(c_, list(r_))

(1402.2626,
 [Craft(item=glacial bag, ingredients=4.0*ebonweave + 4.0*moonshroud + eternium thread),
  AHBuyMarket(item=4.0*ebonweave, gold=728.3392),
  AHBuyMarket(item=4.0*moonshroud, gold=670.4236),
  AHBuyMarket(item=eternium thread, gold=3.4998)])

In [28]:
options = procurement.dnf(pp_market.obtain(i("4 frostweave bag")))
(c_, r_) = min([(procurement.cost(x), x) for x in options], key=lambda x: x[0])
(c_, list(r_))

(269.7264,
 [Craft(item=4*frostweave bag, ingredients=24.0*bolt of imbued frostweave + 8.0*eternium thread),
  Craft(item=24.0*bolt of imbued frostweave, ingredients=48.0*infinite dust + 48.0*bolt of frostweave),
  AHBuyMarket(item=48.0*infinite dust, gold=81.864),
  Craft(item=48.0*bolt of frostweave, ingredients=240.0*frostweave cloth),
  AHBuyMarket(item=240.0*frostweave cloth, gold=159.864),
  AHBuyMarket(item=8.0*eternium thread, gold=27.9984)])

In [29]:
options = procurement.dnf(pp_market.obtain(i("primal might")))
(c_, r_) = min([(procurement.cost(x), x) for x in options], key=lambda x: x[0])
(c_, list(r_))

(52.2975,
 [Craft(item=primal might, ingredients=primal air + primal water + primal fire + primal mana + primal earth),
  Craft(item=primal air, ingredients=10.0*mote of air),
  AHBuyMarket(item=10.0*mote of air, gold=12.238),
  AHBuyMarket(item=primal water, gold=16.7955),
  AHBuyMarket(item=primal fire, gold=4.1164),
  AHBuyMarket(item=primal mana, gold=18.1442),
  AHBuyMarket(item=primal earth, gold=1.0034)])

In [30]:
# drpannkoeken
"Tukui:Profile:fA1smTX1A4X2KeGCrbWxiQ6gPyDlfUcfLIneAuvv8WGnm0Idw2MKI7TPZX2hyg8WCMCMXgiTsXcvv11ErFSd5k1nDtf7URYiu23PSOBdkkBAxDfsTQnTW0ZJXJH0kfVW(4Z57)13)35FMqJ28oaSgOKo06Z2FbdlBGUoWwdzmSukKburkJMv5rnJwOw1AAP3YeI1GgLHsaC8e5sN9V7GeUNxkdQo8th5i2HZuM6pYbtwCpPLkuyPm5wyE5c7S8czbyOHTBVYo3QXBjq(il2DOVyxYcZHyMohQCnRca8Aq75WGnGflXDG7fD7t2PsYt2LzKrJWoccHoQJn(ONn9TlKohb9LiOvIsbj7O96E0f4mAgABamlUtHLYYYicUbKD6w4(Su5rEN89XeecB(pzUDr4Q2Pa2GcWTSLqBvSfptwm9Cu7Ikl(V8oMNnxr4Me7eCpxr3OSJs65n2J888Kf1EYZPH2CaMlZcTd4MI5ANcdj7SyJEpK6kNr9(rIDEpLbpVnGtWfL8J6LRqQRvomeX5hZXKrRmgLbAuRO7LyLT41f(irAzloQWaXOXUFgWttQTHEFZWSk35EcHi)2L5WNc6zmyiEE(GyNVbZG3trGwx8CyreQnUlttszXQoH4DxL(fEcPEEbDXufD7LaKid26Zz8JSyxD9VPfU7f6WHNChohwTr4ybnBPARUQmeubIl6oaV)HJjW7ZVQ3XKfcVqWIx0TpU8X(BdDixZf5)DvIRnh8uL7SalBkv3UuR5eIxQLs(7D0AbEnrBVkk(WFLld6oCfNok5CARPEg9t7sUStxkm8ZjCn6c)SiRoyBioil87qccB5WYIIj)dkJUhpl0HyBRIji6ycSrKfJE8xY8PJ6hCmDXoCUcSwGJQJojgdXcp63OiAXdkaBVDrmXp(cSXKfVPh)sMZ9If(jbAX5XOAM0gDr3X4428784XSQJWUDQJciKUTM5mgLvr4askQNeNKsj0hnpAY5iGwLanZve6MRzs1qGY8hS8cPXye2)QT7i8sLOupK1THIrOYIdAFZ6msBh7yxGlkw3mcl743atbl93iGMic7gJtMh()BeqY3vRcrw0oUovtEssQC0jFsVD)RQSB0w4q0SRWakEnHWh2r3IS4fs8afMTG)Le9SmcCVaozQjcy5)Jq3ESWoP3plgCJRGkgc4cf)0B9R(lpHHlU3p1OZT(ziZBbTJRF5ApH)T(1tYMl6dLlapllAIoH3J)Vp65KumsZ2cAR6eKK5)vSgx5cnNvfGbLTH4XvoxU5RPPxzCLisZJO)2xP8BBzd3yCLEKkOTbCCLl1myO14kx8iIX2SllYAvQanuKOBmCZSilnk3AvVelNMSoMRPI7o2GVBOwlyTSrfu5QWkQ(hKWDSsYqQNEw3dUsOCKgOT6ixHB(u1DNXnkXqxjf3zjSBkzXL98OZdgCLECt)4b5LE84uGJXaQYbY8MB62E6gVmat)YaK4Vaqw89FOFUi7CtbM(Vd(4DY9t5WEOaAfiaT6DNcKMqvnJ1gnr26z7L8v6En7)UiSELmaZ28zqVDKN9vKwAyzXrE2hsuwH3jp02MyT1x721DEABZsBqFvbvZ(xQ06qsSQdzrbIvnhkLMLjr0WVOKfTjetYd1wZuZgLdAc0WQ7txNhQR)21mQQINhAaXa9H33)jX5ld0HF8)WDXFWS)BJWBa0LQzBJmYR9a4nkjJSFh42kMdqhIZpWkleNdT5RygLSh5o)z2nYb(wBcktQJiMdKFtn7YQeOlzWFwQAR8QatyEvTvTvzoEYZ64DykrY(8Wo1lew6Ejo7Ey67jqn4GG8HL(sKPVwdxYN)CV10TyUMjkF8GLweTwgWwPU6ECPpt2wPNCSbYJ6UMYtLj7Ot3DwKoc)MHYEUjU(eVr8iTw2qJpRZ6tAXE1NzQHbwQs0jmQhSSHUMrv)hJOqEtNc8J3JF9M(4tlslIowGFqtQD8L5yBRkrhbnTRUFmNAYiT45hTUQ2Kvb0LFJerFyn6b(nZzriCflLJ8)BBXOsoMZNEpwO5K2RT)S6allTY0bcyLMZqTGkov(Zp"

'Tukui:Profile:fA1smTX1A4X2KeGCrbWxiQ6gPyDlfUcfLIneAuvv8WGnm0Idw2MKI7TPZX2hyg8WCMCMXgiTsXcvv11ErFSd5k1nDtf7URYiu23PSOBdkkBAxDfsTQnTW0ZJXJH0kfVW(4Z57)13)35FMqJ28oaSgOKo06Z2FbdlBGUoWwdzmSukKburkJMv5rnJwOw1AAP3YeI1GgLHsaC8e5sN9V7GeUNxkdQo8th5i2HZuM6pYbtwCpPLkuyPm5wyE5c7S8czbyOHTBVYo3QXBjq(il2DOVyxYcZHyMohQCnRca8Aq75WGnGflXDG7fD7t2PsYt2LzKrJWoccHoQJn(ONn9TlKohb9LiOvIsbj7O96E0f4mAgABamlUtHLYYYicUbKD6w4(Su5rEN89XeecB(pzUDr4Q2Pa2GcWTSLqBvSfptwm9Cu7Ikl(V8oMNnxr4Me7eCpxr3OSJs65n2J888Kf1EYZPH2CaMlZcTd4MI5ANcdj7SyJEpK6kNr9(rIDEpLbpVnGtWfL8J6LRqQRvomeX5hZXKrRmgLbAuRO7LyLT41f(irAzloQWaXOXUFgWttQTHEFZWSk35EcHi)2L5WNc6zmyiEE(GyNVbZG3trGwx8CyreQnUlttszXQoH4DxL(fEcPEEbDXufD7LaKid26Zz8JSyxD9VPfU7f6WHNChohwTr4ybnBPARUQmeubIl6oaV)HJjW7ZVQ3XKfcVqWIx0TpU8X(BdDixZf5)DvIRnh8uL7SalBkv3UuR5eIxQLs(7D0AbEnrBVkk(WFLld6oCfNok5CARPEg9t7sUStxkm8ZjCn6c)SiRoyBioil87qccB5WYIIj)dkJUhpl0HyBRIji6ycSrKfJE8xY8PJ6hCmDXoCUcSwGJQJojgdXcp63OiAXdkaBVDrmXp(cSXKfVPh)sMZ9If(jbAX5XOAM0gDr3X4428784XSQJWUDQJciKUTM5mgLvr4askQNeNKsj0hnpA